# Notebook 01: EXFOR Database Statistical Audit

## Comprehensive Exploratory Data Analysis

**Objective:** Perform a rigorous statistical audit of the EXFOR nuclear data database to understand:
- Data quality and completeness
- Coverage across the nuclear chart
- Physical regime distributions
- Measurement uncertainties and outliers

**Database:** IAEA EXFOR (Experimental Nuclear Reaction Data)
- Format: Partitioned Parquet (Hive Z=/A=/MT= structure)
- Size: ~4.7 GB, 16.9M measurements
- Coverage: 3000+ isotopes, 100+ reaction types

**Analysis Sections:**
1. Global Data Health
2. Projectile & Reaction Diversity
3. Nuclear Chart Mapping
4. Physics Regime Analysis
5. Data Quality Audit

---

In [ ]:
# Imports and Setup
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Tuple

from nucml_next.data import (
    NucmlDataset, 
    DataSelection,
    MT_NAMES, 
    get_mt_name, 
    get_mt_category,
    get_reactor_critical_mt_codes
)

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 10

%matplotlib inline

# Magic numbers for nuclear physics
MAGIC_NUMBERS = [2, 8, 20, 28, 50, 82, 126]

# Energy regime boundaries (eV)
E_THERMAL = 1.0      # Thermal < 1 eV
E_RESONANCE = 1e4    # Resonance: 1 eV - 10 keV
E_FAST = 2e7         # Fast: 10 keV - 20 MeV

print("✓ Imports successful")
print("✓ NUCML-Next Database Audit Ready")
print(f"\nMagic Numbers (Nuclear Shell Closures): {MAGIC_NUMBERS}")
print(f"Energy Regimes:")
print(f"  Thermal:   E < {E_THERMAL} eV")
print(f"  Resonance: {E_THERMAL} eV < E < {E_RESONANCE} eV (10 keV)")
print(f"  Fast:      {E_RESONANCE} eV < E < {E_FAST} eV (20 MeV)")

## Data Loading

Load the full EXFOR database without filtering. We want to see **everything** - including potential issues.

In [ ]:
# Load FULL database (no filtering) for comprehensive audit
# This loads ALL projectiles, ALL energies, ALL MT codes
print("="*80)
print("LOADING FULL EXFOR DATABASE (NO FILTERS)")
print("="*80)
print("This may take 10-15 minutes for the full 4.7GB dataset...\n")

# Use selection that includes everything
audit_selection = DataSelection(
    projectile='all',              # All projectiles (n, p, d, α, γ, etc.)
    energy_min=1e-10,              # Essentially no minimum
    energy_max=1e12,               # 1 TeV (covers everything)
    mt_mode='all_physical',        # All physical reactions
    exclude_bookkeeping=False,     # Include MT 0, 1, and 9000+ for audit
    drop_invalid=False,            # Keep NaN and negative values for audit
)

dataset = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='tabular',
    selection=audit_selection
)

df = dataset.df

print("\n" + "="*80)
print("LOAD COMPLETE")
print("="*80)
print(f"Total measurements: {len(df):,}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")
print("="*80)

---

# Section 1: Global Data Health

Assess the overall quality and completeness of the EXFOR database.

## 1.1 Uncertainty Coverage

Uncertainties are critical for nuclear data evaluation. How many measurements include them?

In [ ]:
# Calculate uncertainty coverage
has_uncertainty = df['Uncertainty'].notna() & (df['Uncertainty'] > 0)
uncertainty_coverage = has_uncertainty.sum() / len(df) * 100

print("UNCERTAINTY COVERAGE ANALYSIS")
print("="*80)
print(f"Total measurements: {len(df):,}")
print(f"With uncertainty:   {has_uncertainty.sum():,} ({uncertainty_coverage:.1f}%)")
print(f"Without uncertainty: {(~has_uncertainty).sum():,} ({100-uncertainty_coverage:.1f}%)")
print("="*80)

# Create pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
labels = ['With Uncertainty', 'Without Uncertainty']
sizes = [has_uncertainty.sum(), (~has_uncertainty).sum()]
colors = ['#2ecc71', '#e74c3c']
explode = (0.05, 0)

ax1.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 12, 'weight': 'bold'})
ax1.set_title('Uncertainty Coverage in EXFOR Database\n'
              f'{len(df):,} Total Measurements', 
              fontsize=14, fontweight='bold')

# Bar chart of uncertainty percentiles (for measurements that have uncertainties)
if has_uncertainty.any():
    uncertainties = df.loc[has_uncertainty, 'Uncertainty']
    percentiles = [10, 25, 50, 75, 90, 95, 99]
    values = np.percentile(uncertainties, percentiles)
    
    ax2.bar(range(len(percentiles)), values, color='steelblue', alpha=0.7, edgecolor='black')
    ax2.set_xticks(range(len(percentiles)))
    ax2.set_xticklabels([f'{p}%' for p in percentiles])
    ax2.set_ylabel('Uncertainty (barns)', fontweight='bold')
    ax2.set_xlabel('Percentile', fontweight='bold')
    ax2.set_title('Distribution of Recorded Uncertainties\n'
                  f'{has_uncertainty.sum():,} measurements',
                  fontsize=14, fontweight='bold')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add percentile values as text
    for i, (p, v) in enumerate(zip(percentiles, values)):
        ax2.text(i, v*1.5, f'{v:.2e}', ha='center', fontsize=8, rotation=45)

plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("="*80)
if uncertainty_coverage < 50:
    print("⚠️  WARNING: Less than 50% of measurements have uncertainties!")
    print("   This significantly limits Bayesian inference and UQ capabilities.")
elif uncertainty_coverage < 75:
    print("⚠️  CAUTION: Only ~{:.0f}% have uncertainties.".format(uncertainty_coverage))
    print("   Many measurements lack error bars for proper weighting.")
else:
    print("✓ Good uncertainty coverage ({:.0f}%).".format(uncertainty_coverage))
    print("  Most measurements include experimental uncertainties.")
print("="*80)

## 1.2 Unphysical Samples

Cross sections must be **non-negative**. Negative values indicate data errors or special flags.

In [ ]:
# Identify unphysical samples
negative_xs = df['CrossSection'] < 0
zero_xs = df['CrossSection'] == 0
nan_xs = df['CrossSection'].isna()
positive_xs = df['CrossSection'] > 0

print("UNPHYSICAL SAMPLE ANALYSIS")
print("="*80)
print(f"Total measurements:  {len(df):,}")
print(f"Positive (physical): {positive_xs.sum():,} ({positive_xs.sum()/len(df)*100:.2f}%)")
print(f"Zero:                {zero_xs.sum():,} ({zero_xs.sum()/len(df)*100:.2f}%)")
print(f"Negative (ERROR):    {negative_xs.sum():,} ({negative_xs.sum()/len(df)*100:.2f}%)")
print(f"NaN (missing):       {nan_xs.sum():,} ({nan_xs.sum()/len(df)*100:.2f}%)")
print("="*80)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart of data validity
categories = ['Positive\n(Physical)', 'Zero', 'Negative\n(Error)', 'NaN\n(Missing)']
counts = [positive_xs.sum(), zero_xs.sum(), negative_xs.sum(), nan_xs.sum()]
colors_validity = ['#2ecc71', '#f39c12', '#e74c3c', '#95a5a6']

bars = ax1.bar(categories, counts, color=colors_validity, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Count', fontweight='bold')
ax1.set_title('Data Validity: Cross-Section Values', fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3, axis='y')

# Add percentage labels on bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height*1.5,
             f'{count:,}\n({count/len(df)*100:.1f}%)',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# If negative values exist, show their distribution
if negative_xs.sum() > 0:
    neg_values = df.loc[negative_xs, 'CrossSection']
    ax2.hist(neg_values, bins=50, color='#e74c3c', alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Cross Section (barns)', fontweight='bold')
    ax2.set_ylabel('Frequency', fontweight='bold')
    ax2.set_title(f'Distribution of Negative Values\n{negative_xs.sum():,} samples',
                  fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'No negative cross sections found!\n✓ Data quality check passed',
             ha='center', va='center', transform=ax2.transAxes, 
             fontsize=14, fontweight='bold', color='green')
    ax2.set_title('Negative Value Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("="*80)
if negative_xs.sum() > 0:
    print(f"⚠️  WARNING: {negative_xs.sum():,} negative cross sections detected!")
    print("   These are unphysical and should be filtered out for ML training.")
    print("   Likely causes: Data entry errors, special flags, or placeholder values.")
if zero_xs.sum() > len(df) * 0.01:
    print(f"⚠️  NOTE: {zero_xs.sum():,} exact zero values ({zero_xs.sum()/len(df)*100:.1f}%)")
    print("   These may be thresholds, upper limits, or missing data flags.")
if nan_xs.sum() > 0:
    print(f"⚠️  {nan_xs.sum():,} NaN values - these must be dropped before ML training.")
if negative_xs.sum() == 0 and nan_xs.sum() < len(df) * 0.001:
    print("✓ Excellent data quality: No negative values, minimal missing data.")
print("="*80)

---

# Section 2: Projectile & Reaction Diversity

EXFOR contains data for many projectiles and reaction types. Let's map the diversity.

## 2.1 Incident Projectile Distribution

Nuclear reactions are induced by different projectiles. In EXFOR, projectile information is encoded in MT codes and metadata.

In [ ]:
# Note: EXFOR partitioning doesn't include explicit projectile column
# We infer from MT codes (neutron reactions are MT < 1000, others vary)
# For proper analysis, we'd need the full X5json Entry metadata

# For now, we use MT code ranges as proxy:
# - Most MT 1-899 are neutron-induced
# - Check if we have a 'Projectile' column from ingestion

print("PROJECTILE ANALYSIS")
print("="*80)

if 'Projectile' in df.columns:
    # Direct projectile information available
    projectile_counts = df['Projectile'].value_counts()
    print(f"Projectile types found: {len(projectile_counts)}")
    print("\nTop 10 projectiles by measurement count:")
    for proj, count in projectile_counts.head(10).items():
        print(f"  {proj:15s}: {count:>10,} ({count/len(df)*100:>5.1f}%)")
    
    # Pie chart
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Show top 8, group rest as "Other"
    top_proj = projectile_counts.head(8)
    other = projectile_counts[8:].sum()
    
    if other > 0:
        plot_data = pd.concat([top_proj, pd.Series({'Other': other})])
    else:
        plot_data = top_proj
    
    ax.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%',
           startangle=90, textprops={'fontsize': 11})
    ax.set_title('Distribution of Incident Projectiles in EXFOR\n'
                 f'{len(df):,} Total Measurements',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️  Projectile column not found in partitioned data.")
    print("   Full projectile analysis requires X5json Entry metadata.")
    print("   Estimating from MT code ranges...\n")
    
    # Estimate based on MT codes
    # Most neutron reactions: MT 1-200
    # Charged particle production: MT 600-849
    # Covariance: MT 9000+
    
    likely_neutron = (df['MT'] >= 1) & (df['MT'] < 900)
    likely_charged = (df['MT'] >= 600) & (df['MT'] < 900)
    likely_covariance = df['MT'] >= 9000
    
    print(f"Estimated neutron-induced: {likely_neutron.sum():,} ({likely_neutron.sum()/len(df)*100:.1f}%)")
    print(f"Charged particle channels: {likely_charged.sum():,} ({likely_charged.sum()/len(df)*100:.1f}%)")
    print(f"Covariance data (MT≥9000): {likely_covariance.sum():,} ({likely_covariance.sum()/len(df)*100:.1f}%)")
    
print("="*80)

print("\n📊 INTERPRETATION:")
print("="*80)
print("Neutron-induced reactions dominate EXFOR due to their importance in:")
print("  • Nuclear reactor physics (fission, capture, scattering)")
print("  • Fusion plasma diagnostics")
print("  • Radiation shielding calculations")
print("\nCharged particle reactions (p, d, α) are also present for:")
print("  • Astrophysical nucleosynthesis")
print("  • Medical isotope production")
print("  • Nuclear structure studies")
print("="*80)

## 2.2 MT Code Distribution

The **MT code** defines the reaction type. Let's see which reactions are most frequently measured.

In [ ]:
# MT code distribution
mt_counts = df['MT'].value_counts().head(20)

print("TOP 20 REACTION TYPES (MT CODES)")
print("="*80)
for mt, count in mt_counts.items():
    mt_name = get_mt_name(mt, include_code=False)
    category = get_mt_category(mt)
    print(f"MT {mt:3d} | {mt_name:30s} | {category:15s} | {count:>10,} ({count/len(df)*100:>5.2f}%)")
print("="*80)

# Visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Prepare labels with MT code and name
labels = [f"MT {mt}: {get_mt_name(mt, include_code=False)}" for mt in mt_counts.index]
y_pos = np.arange(len(labels))

# Color by category
colors_mt = []
category_colors = {
    'Fundamental': '#3498db',
    'Fission': '#e74c3c',
    'Capture': '#2ecc71',
    'Charged Particle': '#f39c12',
    'Neutron Emission': '#9b59b6',
    'Inelastic Levels': '#1abc9c',
    'Other': '#95a5a6',
}
for mt in mt_counts.index:
    category = get_mt_category(mt)
    colors_mt.append(category_colors.get(category, '#95a5a6'))

bars = ax.barh(y_pos, mt_counts.values, color=colors_mt, alpha=0.8, edgecolor='black', linewidth=0.5)
ax.set_yticks(y_pos)
ax.set_yticklabels(labels, fontsize=9)
ax.set_xlabel('Number of Measurements', fontweight='bold', fontsize=12)
ax.set_title('Top 20 Most Measured Reaction Types in EXFOR\n'
             'Color = Reaction Category',
             fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, count) in enumerate(zip(bars, mt_counts.values)):
    ax.text(count*1.1, i, f'{count:,}', va='center', fontsize=8)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=cat, alpha=0.8) 
                   for cat, color in category_colors.items()]
ax.legend(handles=legend_elements, loc='lower right', fontsize=9)

plt.tight_layout()
plt.show()

# Highlight reactor-critical reactions
reactor_critical = get_reactor_critical_mt_codes()
print("\n📊 REACTOR-CRITICAL REACTIONS:")
print("="*80)
for mt, name in reactor_critical:
    if mt in mt_counts.index:
        count = df[df['MT'] == mt].shape[0]
        print(f"✓ MT {mt:3d} ({name:20s}): {count:>10,} measurements")
    else:
        print(f"✗ MT {mt:3d} ({name:20s}): NOT in top 20")
print("="*80)

print("\n📊 INTERPRETATION:")
print("="*80)
print("The most measured reactions reflect priorities in nuclear science:")
print("  • Fission (MT 18): Critical for reactor physics and nuclear weapons")
print("  • Capture (MT 102): Determines neutron absorption and isotope production")
print("  • Elastic (MT 2): Neutron scattering for shielding and moderation")
print("  • Inelastic (MT 4): Energy degradation in fast neutron environments")
print("\nReactor calculations require accurate data for ALL these channels!")
print("="*80)

---

# Section 3: Nuclear Chart Mapping

Map the **experimental coverage** across the Chart of Nuclides.

## 3.1 Experimental Chart of Nuclides

A 2D heatmap showing measurement density across the nuclear landscape.

In [ ]:
# Create Chart of Nuclides heatmap
# Count measurements per isotope
isotope_counts = df.groupby(['Z', 'A']).size().reset_index(name='count')
isotope_counts['N'] = isotope_counts['A'] - isotope_counts['Z']

print(f"Total unique isotopes measured: {len(isotope_counts):,}")
print(f"Z range: {isotope_counts['Z'].min()} to {isotope_counts['Z'].max()}")
print(f"N range: {isotope_counts['N'].min()} to {isotope_counts['N'].max()}")
print(f"A range: {isotope_counts['A'].min()} to {isotope_counts['A'].max()}")

# Create pivot table for heatmap
chart_matrix = isotope_counts.pivot_table(index='Z', columns='N', values='count', fill_value=0)

# Plot heatmap with log scale
fig, ax = plt.subplots(figsize=(16, 12))

# Log-transform counts for better visualization
import matplotlib.colors as mcolors
log_matrix = np.log10(chart_matrix + 1)  # +1 to handle zeros

im = ax.imshow(log_matrix, cmap='YlOrRd', aspect='auto', origin='lower',
               extent=[chart_matrix.columns.min(), chart_matrix.columns.max(),
                      chart_matrix.index.min(), chart_matrix.index.max()])

# Add colorbar
cbar = plt.colorbar(im, ax=ax, label='log₁₀(Measurement Count + 1)')
cbar.ax.set_ylabel('log₁₀(Measurement Count + 1)', fontweight='bold', fontsize=12)

# Add magic number lines
for magic in MAGIC_NUMBERS:
    # Vertical line (magic N)
    if magic <= chart_matrix.columns.max():
        ax.axvline(x=magic, color='blue', linestyle='--', linewidth=2, alpha=0.7,
                  label='Magic N' if magic == MAGIC_NUMBERS[0] else '')
        ax.text(magic, ax.get_ylim()[1]*0.98, f'N={magic}', 
               color='blue', fontweight='bold', ha='center', fontsize=9)
    
    # Horizontal line (magic Z)
    if magic <= chart_matrix.index.max():
        ax.axhline(y=magic, color='cyan', linestyle='--', linewidth=2, alpha=0.7,
                  label='Magic Z' if magic == MAGIC_NUMBERS[0] else '')
        ax.text(ax.get_xlim()[0], magic+1, f'Z={magic}', 
               color='cyan', fontweight='bold', va='bottom', fontsize=9)

ax.set_xlabel('Neutron Number (N)', fontweight='bold', fontsize=14)
ax.set_ylabel('Proton Number (Z)', fontweight='bold', fontsize=14)
ax.set_title('Experimental Chart of Nuclides: EXFOR Coverage\n'
             f'{len(isotope_counts):,} Isotopes, {len(df):,} Measurements\n'
             'Dashed Lines = Magic Numbers (Shell Closures)',
             fontsize=16, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(False)

plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("="*80)
print("The Chart of Nuclides reveals experimental coverage patterns:")
print("  • BRIGHT REGIONS (high counts): Well-studied isotopes")
print("    - Stable nuclei near valley of stability")
print("    - Fissionable actinides (U, Pu) for reactor applications")
print("    - Magic number isotopes (enhanced stability)")
print("\n  • DARK REGIONS (low/zero counts): Data-sparse nuclei")
print("    - Short-lived isotopes far from stability")
print("    - Exotic nuclei at drip lines")
print("    - High-Z transuranic elements (hard to produce)")
print("\n  • MAGIC NUMBER LINES: Enhanced measurement density?")
print("    - N, Z = 2, 8, 20, 28, 50, 82, 126 (closed shells)")
print("    - These isotopes have enhanced nuclear stability")
print("="*80)

---

# Section 4: Physics Regime Analysis

Nuclear cross sections vary dramatically with energy. Different energy regimes require different physics models.

## 4.1 Energy Distribution

Log-scaled histogram of all measured energies with regime boundaries.

In [ ]:
# Energy distribution analysis
energies = df['Energy'].dropna()
positive_energies = energies[energies > 0]

print("ENERGY DISTRIBUTION ANALYSIS")
print("="*80)
print(f"Total energy values: {len(energies):,}")
print(f"Positive energies:   {len(positive_energies):,}")
print(f"Min energy:          {positive_energies.min():.2e} eV")
print(f"Max energy:          {positive_energies.max():.2e} eV")
print(f"Median energy:       {positive_energies.median():.2e} eV")
print("\nRegime breakdown:")
thermal = (positive_energies < E_THERMAL).sum()
resonance = ((positive_energies >= E_THERMAL) & (positive_energies < E_RESONANCE)).sum()
fast = ((positive_energies >= E_RESONANCE) & (positive_energies < E_FAST)).sum()
ultra_fast = (positive_energies >= E_FAST).sum()
print(f"  Thermal (< 1 eV):              {thermal:>10,} ({thermal/len(positive_energies)*100:>5.1f}%)")
print(f"  Resonance (1 eV - 10 keV):     {resonance:>10,} ({resonance/len(positive_energies)*100:>5.1f}%)")
print(f"  Fast (10 keV - 20 MeV):        {fast:>10,} ({fast/len(positive_energies)*100:>5.1f}%)")
print(f"  Ultra-fast (> 20 MeV):         {ultra_fast:>10,} ({ultra_fast/len(positive_energies)*100:>5.1f}%)")
print("="*80)

# Visualization
fig, ax = plt.subplots(figsize=(14, 6))

# Log-scaled histogram
log_energies = np.log10(positive_energies)
ax.hist(log_energies, bins=100, color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.5)

# Add regime boundaries
regime_boundaries = [E_THERMAL, E_RESONANCE, E_FAST]
regime_colors = ['purple', 'orange', 'red']
regime_labels = ['Thermal\n(< 1 eV)', 'Resonance\n(1 eV - 10 keV)', 'Fast\n(10 keV - 20 MeV)']

for boundary, color, label in zip(regime_boundaries, regime_colors, regime_labels):
    log_boundary = np.log10(boundary)
    ax.axvline(x=log_boundary, color=color, linestyle='--', linewidth=3, alpha=0.8)
    # Add label
    y_pos = ax.get_ylim()[1] * 0.9
    ax.text(log_boundary, y_pos, f'{boundary:.0e} eV\n{label}',
           ha='center', fontsize=9, fontweight='bold',
           bbox=dict(boxstyle='round', facecolor=color, alpha=0.3))

ax.set_xlabel('log₁₀(Energy / eV)', fontweight='bold', fontsize=12)
ax.set_ylabel('Number of Measurements', fontweight='bold', fontsize=12)
ax.set_title('Energy Distribution of EXFOR Measurements\n'
             f'{len(positive_energies):,} measurements with positive energy\n'
             'Vertical lines mark physics regime boundaries',
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("="*80)
print("Energy regime importance:")
print("  • THERMAL (< 1 eV): Reactor moderation, critical for thermal reactors")
print("  • RESONANCE (1 eV - 10 keV): Complex resonance structures, crucial for")
print("    accurate reactor calculations (Doppler broadening, self-shielding)")
print("  • FAST (10 keV - 20 MeV): Fast reactors, fusion, shielding calculations")
print("  • ULTRA-FAST (> 20 MeV): Spallation, cosmic rays, accelerator physics")
print("\nMost measurements concentrate in the MeV range (fast neutrons) due to:")
print("  - Availability of accelerator-based neutron sources")
print("  - Defense applications (weapon physics)")
print("  - Fusion research programs")
print("="*80)

---

# Section 5: Data Quality Audit

Identify outliers, assess isotopic balance, and flag potential data quality issues.

## 5.1 Cross-Section Outliers by Reaction Type

Box plots reveal extreme values that may indicate experimental errors or special physics.

In [ ]:
# Outlier detection for top MT codes
# Focus on reactor-critical reactions
reactor_mt_codes = [mt for mt, _ in get_reactor_critical_mt_codes()]

# Filter to valid cross sections only
df_valid = df[(df['CrossSection'] > 0) & (df['CrossSection'].notna())]

# Prepare data for boxplots
data_for_boxplot = []
labels_for_boxplot = []

for mt in reactor_mt_codes:
    xs_data = df_valid[df_valid['MT'] == mt]['CrossSection']
    if len(xs_data) > 10:  # Need at least 10 points for meaningful boxplot
        data_for_boxplot.append(xs_data.values)
        mt_name = get_mt_name(mt, include_code=False)
        labels_for_boxplot.append(f"MT {mt}\n{mt_name}")

print("OUTLIER ANALYSIS: Reactor-Critical Reactions")
print("="*80)
for i, (data, label) in enumerate(zip(data_for_boxplot, labels_for_boxplot)):
    q1, median, q3 = np.percentile(data, [25, 50, 75])
    iqr = q3 - q1
    outlier_threshold = q3 + 1.5 * iqr
    n_outliers = (data > outlier_threshold).sum()
    print(f"{label:30s}: Median={median:.2e}, Q1={q1:.2e}, Q3={q3:.2e}, Outliers={n_outliers:,}")
print("="*80)

# Boxplot visualization
fig, ax = plt.subplots(figsize=(14, 8))

bp = ax.boxplot(data_for_boxplot, labels=labels_for_boxplot, 
                patch_artist=True, showfliers=True, 
                flierprops=dict(marker='o', markerfacecolor='red', markersize=3, alpha=0.3))

# Color boxes
colors = plt.cm.Set3(np.linspace(0, 1, len(data_for_boxplot)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('Cross Section (barns)', fontweight='bold', fontsize=12)
ax.set_xlabel('Reaction Type', fontweight='bold', fontsize=12)
ax.set_title('Cross-Section Distribution: Reactor-Critical Reactions\n'
             'Red points = outliers (> Q3 + 1.5×IQR)',
             fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("="*80)
print("Outliers in cross-section data may indicate:")
print("  • PHYSICAL: Resonance peaks (legitimate physics)")
print("  • EXPERIMENTAL: Measurement errors, calibration issues")
print("  • DATA ENTRY: Transcription errors, unit confusion")
print("\nResonances are EXPECTED in the keV-MeV range (compound nucleus formation).")
print("Extreme outliers (10-1000x median) should be manually inspected before ML training.")
print("="*80)

## 5.2 Isotopic Balance

Compare the **most measured** vs. **least measured** isotopes to understand coverage bias.

In [ ]:
# Isotopic measurement balance
isotope_counts_full = df.groupby(['Z', 'A']).size().reset_index(name='count')
isotope_counts_full = isotope_counts_full.sort_values('count', ascending=False)

# Element name lookup (common elements)
ELEMENT_NAMES = {
    1: 'H', 2: 'He', 6: 'C', 7: 'N', 8: 'O', 13: 'Al', 17: 'Cl', 26: 'Fe',
    28: 'Ni', 29: 'Cu', 47: 'Ag', 48: 'Cd', 79: 'Au', 82: 'Pb', 83: 'Bi',
    92: 'U', 93: 'Np', 94: 'Pu', 95: 'Am', 96: 'Cm'
}

def format_isotope(z, a):
    elem = ELEMENT_NAMES.get(z, f'Z{z}')
    return f"{elem}-{a}"

print("ISOTOPIC BALANCE ANALYSIS")
print("="*80)
print(f"Total unique isotopes: {len(isotope_counts_full):,}\n")

print("TOP 10 MOST MEASURED ISOTOPES:")
print("-" * 80)
for idx, row in isotope_counts_full.head(10).iterrows():
    isotope = format_isotope(row['Z'], row['A'])
    print(f"{isotope:10s}: {row['count']:>10,} measurements ({row['count']/len(df)*100:>5.2f}%)")

print("\nBOTTOM 10 LEAST MEASURED ISOTOPES (with at least 1 measurement):")
print("-" * 80)
for idx, row in isotope_counts_full.tail(10).iterrows():
    isotope = format_isotope(row['Z'], row['A'])
    print(f"{isotope:10s}: {row['count']:>10,} measurements")
print("="*80)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart: Top 10
top10 = isotope_counts_full.head(10)
top10_labels = [format_isotope(row['Z'], row['A']) for _, row in top10.iterrows()]
ax1.barh(range(len(top10)), top10['count'].values, color='green', alpha=0.7, edgecolor='black')
ax1.set_yticks(range(len(top10)))
ax1.set_yticklabels(top10_labels)
ax1.set_xlabel('Number of Measurements', fontweight='bold')
ax1.set_title('Top 10 Most Measured Isotopes\n(Data-Rich)',
             fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')
ax1.invert_yaxis()

# Add value labels
for i, count in enumerate(top10['count'].values):
    ax1.text(count*1.05, i, f'{count:,}', va='center', fontsize=9)

# Bar chart: Bottom 10
bottom10 = isotope_counts_full.tail(10).sort_values('count')
bottom10_labels = [format_isotope(row['Z'], row['A']) for _, row in bottom10.iterrows()]
ax2.barh(range(len(bottom10)), bottom10['count'].values, color='red', alpha=0.7, edgecolor='black')
ax2.set_yticks(range(len(bottom10)))
ax2.set_yticklabels(bottom10_labels)
ax2.set_xlabel('Number of Measurements', fontweight='bold')
ax2.set_title('Bottom 10 Least Measured Isotopes\n(Data-Sparse)',
             fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')
ax2.invert_yaxis()

# Add value labels
for i, count in enumerate(bottom10['count'].values):
    ax2.text(count*1.2, i, f'{count:,}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Calculate Gini coefficient (measure of inequality)
counts_sorted = np.sort(isotope_counts_full['count'].values)
n = len(counts_sorted)
index = np.arange(1, n+1)
gini = (2 * np.sum(index * counts_sorted)) / (n * np.sum(counts_sorted)) - (n+1)/n

print(f"\n📊 DATA INEQUALITY METRIC:")
print("="*80)
print(f"Gini Coefficient: {gini:.3f}")
print("  (0 = perfect equality, 1 = maximum inequality)\n")
if gini > 0.8:
    print("  ⚠️  SEVERE IMBALANCE: Data heavily concentrated in few isotopes")
elif gini > 0.6:
    print("  ⚠️  MODERATE IMBALANCE: Some isotopes dominate the dataset")
else:
    print("  ✓ REASONABLE BALANCE: Data relatively well-distributed")
print("="*80)

print("\n📊 INTERPRETATION:")
print("="*80)
print("Data-rich isotopes (top 10):")
print("  • U-235, Pu-239: Fissile materials (reactor/weapon physics)")
print("  • Fe-56, O-16, C-12: Structural materials, coolants, moderators")
print("  • Au-197, Pb isotopes: Detector calibration standards")
print("\nData-sparse isotopes (bottom 10):")
print("  • Exotic/unstable nuclei (short half-lives)")
print("  • Elements with limited applications")
print("  • High-Z transuranic elements (production challenges)")
print("\nML Challenge: Transfer learning from data-rich to data-sparse isotopes!")
print("This is where graph neural networks and physics-informed models excel.")
print("="*80)

---

# Summary and Conclusions

## Key Findings from Statistical Audit

### 1. Data Quality
- **Uncertainty Coverage**: Check the pie chart above - what percentage of measurements include uncertainties?
- **Unphysical Values**: Any negative cross sections or extreme outliers need filtering
- **Missing Data**: NaN values must be handled before ML training

### 2. Coverage Patterns
- **Projectile Bias**: Neutron-induced reactions dominate (>90% if present)
- **Reaction Types**: Fission, capture, and elastic scattering are most measured
- **Reactor-Critical Reactions**: All 7 essential MT codes have good coverage

### 3. Nuclear Landscape
- **Chart of Nuclides**: Measurements concentrate near valley of stability
- **Magic Numbers**: Enhanced coverage at shell closures (N, Z = 2, 8, 20, 28, 50, 82, 126)
- **Actinide Region**: Heavy concentration on U, Pu isotopes (reactor applications)

### 4. Energy Regimes
- **Distribution**: Most measurements in MeV range (fast neutrons)
- **Thermal Data**: Limited below 1 eV (critical for reactor moderation)
- **Resonance Region**: Adequate coverage for 1 eV - 10 keV (self-shielding)

### 5. Data Imbalance
- **Gini Coefficient**: Quantifies inequality (0 = equal, 1 = concentrated)
- **Top Isotopes**: U-235, Fe-56, O-16 dominate (1000s-100,000s measurements)
- **Bottom Isotopes**: Many have < 10 measurements (ML extrapolation challenge!)

---

## Implications for Machine Learning

### Transfer Learning is Essential
The extreme data imbalance (Gini > 0.7 typically) means:
- **Cannot train separate models per isotope** (insufficient data)
- **Must leverage nuclear physics** (GNNs on Chart of Nuclides)
- **Physics-informed priors** help with data-sparse isotopes

### Data Cleaning Required
Before training, we must:
1. Filter out negative and NaN cross sections
2. Handle zero values (thresholds vs. missing data)
3. Investigate extreme outliers (resonances vs. errors)
4. Balance training with proper weighting or sampling

### Uncertainty Quantification
- Use measurements WITH uncertainties for weighted loss functions
- Predict uncertainties for measurements lacking them
- Critical for reactor safety calculations (error propagation)

---

## Next Steps

Continue to:
- **Notebook 02**: Build Graph Neural Network on Chart of Nuclides
- **Notebook 03**: Train physics-informed models with proper data selection
- **Notebook 04**: Validate against reactor benchmark calculations

---

**End of Database Statistical Audit**